<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/quantum_intelligence_tfq_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set up TFQ-compatible packages
!pip install tensorflow==2.12.0 tensorflow-quantum==0.7.3 cirq==1.2.0

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import json
import math
import argparse
from typing import Tuple, List

import numpy as np

# Fail early with a helpful message if TFQ is unavailable.
try:
    import tensorflow as tf
    import tensorflow_quantum as tfq
    import cirq
    import sympy
except Exception as e:
    print("\n[ERROR] Could not import required libraries.")
    print("Make sure you installed compatible versions and are in the right env.")
    print("Try: bash setup_tfq_env.sh, then: source tfq-env/bin/activate\n")
    raise

# ---------------------------
# Reproducibility
# ---------------------------
def set_seed(seed: int = 42):
    np.random.seed(seed)
    tf.random.set_seed(seed)


# ---------------------------
# Quantum model construction
# ---------------------------
def build_parametric_circuit(n_qubits: int = 4):
    """
    Returns:
      qubits: List[cirq.Qid]
      circuit: cirq.Circuit with parameterized rotations and light entanglement
      symbols: List[sympy.Symbol] of feature symbols
      readout_ops: List[cirq.PauliSumLike] Z expectations on each qubit
    """
    qubits = cirq.GridQubit.rect(1, n_qubits)
    symbols = list(sympy.symbols(f"x0:{n_qubits}"))

    circuit = cirq.Circuit()

    # Feature encoding and mixing
    for i, q in enumerate(qubits):
        circuit.append(cirq.H(q))
        circuit.append(cirq.rz(symbols[i])(q))
        circuit.append(cirq.rx(symbols[i] / 2.0)(q))

    # Light entanglement: CZ ring
    for i in range(n_qubits - 1):
        circuit.append(cirq.CZ(qubits[i], qubits[i + 1]))
    if n_qubits > 2:
        circuit.append(cirq.CZ(qubits[-1], qubits[0]))

    readout_ops = [cirq.Z(q) for q in qubits]
    return qubits, circuit, symbols, readout_ops


def make_model(n_qubits: int = 4,
               hidden: int = 32,
               n_classes: int = 4,
               lr: float = 1e-3) -> Tuple[tf.keras.Model, List[sympy.Symbol]]:
    """
    Build a TFQ model:
      Inputs: [circuits_tensor, symbol_values]
      Output: class probabilities
    """
    _, model_circuit, symbols, readouts = build_parametric_circuit(n_qubits)

    circuits_in = tf.keras.Input(shape=(), dtype=tf.dtypes.string, name="circuits")
    values_in = tf.keras.Input(shape=(len(symbols),), dtype=tf.float32, name="symbol_values")

    pqc_out = tfq.layers.PQC(model_circuit, operators=readouts)([circuits_in, values_in])
    x = tf.keras.layers.Dense(hidden, activation="relu")(pqc_out)
    x = tf.keras.layers.Dense(hidden, activation="relu")(x)
    logits = tf.keras.layers.Dense(n_classes, activation="softmax")(x)

    model = tf.keras.Model(inputs=[circuits_in, values_in], outputs=logits)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model, symbols


# ---------------------------
# Data generation
# ---------------------------
def generate_data(n: int = 2000, n_features: int = 4, seed: int = 42):
    """
    Create learnable labels from smooth trigonometric combinations of features.
    Features are angles in [0, 2π], matching rotation-like parameterization.
    """
    rng = np.random.default_rng(seed)
    X = (2.0 * math.pi) * rng.random((n, n_features), dtype=np.float32)

    # Deterministic labels by choosing the strongest signal across four channels
    channels = np.stack([
        np.sin(X[:, 0]) + np.cos(X[:, 1]),
        np.sin(X[:, 1]) + np.cos(X[:, 2]),
        np.sin(X[:, 2]) + np.cos(X[:, 3]),
        np.sin(X[:, 3]) + np.cos(X[:, 0]),
    ], axis=1)
    y = np.argmax(channels, axis=1).astype(np.int32)
    return X, y


def to_tfq_inputs(n_samples: int, base_circuit: cirq.Circuit):
    """
    Convert a base circuit into a batch of identical circuits.
    Each example differs only by its symbol values (classical features).
    """
    return tfq.convert_to_tensor([base_circuit] * n_samples)


# ---------------------------
# Utilities
# ---------------------------
def parse_args():
    p = argparse.ArgumentParser(description="End-to-end TFQ classifier training.")
    p.add_argument("--samples", type=int, default=2000)
    p.add_argument("--epochs", type=int, default=20)
    p.add_argument("--batch-size", type=int, default=64)
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--lr", type=float, default=1e-3)
    p.add_argument("--hidden", type=int, default=32)
    p.add_argument("--val-ratio", type=float, default=0.2)
    p.add_argument("--outdir", type=str, default="tfq_runs")
    return p.parse_args()


def main():
    args = parse_args()
    os.makedirs(args.outdir, exist_ok=True)
    set_seed(args.seed)

    # Build model and data
    model, symbols = make_model(
        n_qubits=len(symbols := list(sympy.symbols(f"x0:{4}"))) if False else 4,
        hidden=args.hidden,
        lr=args.lr
    )
    n_features = 4  # must match circuit symbols count

    # Rebuild base circuit to guarantee consistency with n_features
    _, base_circuit, _, _ = build_parametric_circuit(n_qubits=n_features)

    X, y = generate_data(n=args.samples, n_features=n_features, seed=args.seed)
    n_val = max(1, int(args.val_ratio * args.samples))
    X_train, y_train = X[:-n_val], y[:-n_val]
    X_val, y_val = X[-n_val:], y[-n_val:]

    circuits_train = to_tfq_inputs(len(X_train), base_circuit)
    circuits_val = to_tfq_inputs(len(X_val), base_circuit)

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy", mode="max", patience=5, restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(args.outdir, "best_model.keras"),
            monitor="val_accuracy", mode="max", save_best_only=True
        )
    ]

    history = model.fit(
        x=[circuits_train, X_train],
        y=y_train,
        validation_data=([circuits_val, X_val], y_val),
        epochs=args.epochs,
        batch_size=args.batch_size,
        verbose=1,
        callbacks=callbacks
    )

    val_loss, val_acc = model.evaluate([circuits_val, X_val], y_val, verbose=0)
    print(f"Validation — loss: {val_loss:.4f}, acc: {val_acc:.4f}")

    # Predict on a small random batch
    X_test = (2.0 * math.pi) * np.random.random((8, n_features)).astype(np.float32)
    circuits_test = to_tfq_inputs(len(X_test), base_circuit)
    probs = model.predict([circuits_test, X_test], verbose=0)
    preds = np.argmax(probs, axis=1)
    print("Sample predictions:", preds.tolist())

    # Save run summary and final model
    summary = {
        "config": vars(args),
        "val_loss": float(val_loss),
        "val_acc": float(val_acc),
    }
    with open(os.path.join(args.outdir, "summary.json"), "w") as f:
        json.dump(summary, f, indent=2)
    model.save(os.path.join(args.outdir, "model_last.keras"))
    print(f"Artifacts saved to: {args.outdir}")


if __name__ == "__main__":
    main()